In [23]:
import numpy as np
import tensorflow as tf
import pandas as pd
!pip install tensorflow_addons
import tensorflow_addons as tfa

In [24]:
lis = [0,0,0,1,1,1,0]
lis = [7.8277349e-02, 4.1263926e-01, 1.8007842e-08, 3.1537157e-02, 5.3680845e-04,
  1.1584072e-02, 8.8135757e-02, 1.7259128e-02, 4.0003050e-01]
max = lis[np.argmax(lis)]
ret = [np.argmax(lis)]

In [25]:
lis[np.argmax(lis)]=0

In [26]:
lis[np.argmax(lis)]

0.4000305

In [27]:
max*0.9

0.371375334

In [28]:
while lis[np.argmax(lis)]>max*0.9:
  ret+=[np.argmax(lis)]
  lis[np.argmax(lis)]=0
ret

[1, 8]

In [29]:
','.join(map(str, ret))

'1,8'

In [30]:
def tostr(ser):
  max = lis[np.argmax(lis)]
  ret = [np.argmax(lis)]
  lis[np.argmax(lis)]=0
  while lis[np.argmax(lis)]>max*0.9:
    ret+=[np.argmax(lis)]
    lis[np.argmax(lis)]=0
  return ','.join(map(str, ret))
lis = [7.8277349e-02, 4.4263926e-01, 1.8007842e-08, 3.1537157e-02, 5.3680845e-04,
  1.1584072e-02, 8.8135757e-02, 1.7259128e-02, 4.0003050e-01]
tostr(lis)

'1,8'

In [31]:
from nltk.tokenize import word_tokenize
from nltk import FreqDist

import nltk


nltk.download('punkt')

rewiews = pd.read_csv('/content/HeadHunter_train_no_multi.csv', sep = ';',error_bad_lines = False)
rewiews = rewiews.fillna('')
positives_data = rewiews['positive']
negatives_data = rewiews['negative']
all_words=' '.join(positives_data)
all_words=all_words+' '+' '.join(negatives_data)
all_words=word_tokenize(all_words)
dist=FreqDist(all_words)
num_unique_word=len(dist)
num_unique_word

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


80873

In [32]:
from keras.preprocessing.text import Tokenizer
VOCAB_SIZE = 80000 
tokenizer = Tokenizer(num_words=VOCAB_SIZE)
tokenizer.fit_on_texts(list(positives_data)+list(negatives_data))
Posi_train = tokenizer.texts_to_sequences(positives_data)
Nega_train = tokenizer.texts_to_sequences(negatives_data)

In [33]:
city_data = rewiews['city']
all_words=' '.join(city_data)
all_words=word_tokenize(all_words)
dist=FreqDist(all_words)
VOCAB_SIZE_CITIES =len(dist)

position_data = rewiews['position']
all_words=' '.join(position_data)
all_words=word_tokenize(all_words)
dist=FreqDist(all_words)
VOCAB_SIZE_POSITIONS =len(dist)

In [34]:
tokenizer = Tokenizer(num_words=VOCAB_SIZE_CITIES)
tokenizer.fit_on_texts(list(city_data))
City_train = tokenizer.texts_to_sequences(city_data)


tokenizer = Tokenizer(num_words=VOCAB_SIZE_POSITIONS)
tokenizer.fit_on_texts(list(position_data))
Position_train = tokenizer.texts_to_sequences(position_data)

In [35]:
dasdas=[]
for ex in Position_train:
  dasdas+=[len(ex)]
np.max(dasdas)

35

In [36]:
from keras.preprocessing import sequence,text
MAX_SEQ_LEN = 100
posi_train_data = sequence.pad_sequences(Posi_train, 
                                    padding='post',
                                    maxlen=MAX_SEQ_LEN)
nega_train_data = sequence.pad_sequences(Nega_train, 
                                    padding='post',
                                    maxlen=MAX_SEQ_LEN)

city_train_data = sequence.pad_sequences(City_train, 
                                    padding='post',
                                    maxlen=5)
position_train_data = sequence.pad_sequences(Position_train, 
                                    padding='post',
                                    maxlen=35)



In [37]:
scores_train_data = rewiews[['salary_rating','team_rating','managment_rating','career_rating','workplace_rating','rest_recovery_rating']]
#scores_train_data
scores_train_data = np.array(scores_train_data)
scores_train_data

array([[3, 5, 2, 4, 4, 5],
       [1, 5, 3, 3, 5, 2],
       [1, 4, 1, 1, 1, 2],
       ...,
       [2, 1, 3, 2, 2, 1],
       [4, 5, 1, 1, 1, 1],
       [5, 5, 5, 4, 5, 5]])

In [38]:
rewiews['target']

0        4
1        5
2        1
3        7
4        6
        ..
48111    8
48112    8
48113    8
48114    1
48115    0
Name: target, Length: 48116, dtype: int64

In [39]:
def target_list(ser):
 # print(ser)
  rez = np.zeros(9)
  rez[int(ser)] = 1
  #for ind in ser.split(","):
  #  rez[int(ind)] = 1
  return rez.astype(int)
target = rewiews['target'].apply(target_list)
target = np.array(target.to_list())
target

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 1],
       [0, 1, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]])

In [40]:
class Cities(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.embedding = tf.keras.layers.Embedding(VOCAB_SIZE_CITIES,8,input_length = 5)
        self.lstm = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32,dropout=0.5, recurrent_dropout=0.0,return_sequences=False))

    def call(self, inp, is_training=False):
        out = self.embedding(inp)
        out = self.lstm(out)
        return out

class Positions(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.embedding = tf.keras.layers.Embedding(VOCAB_SIZE_POSITIONS,16,input_length = 35)
        self.lstm = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32,dropout=0.5, recurrent_dropout=0.0,return_sequences=False))

    def call(self, inp, is_training=False):
        out = self.embedding(inp)
        out = self.lstm(out)
        return out

class Positives(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.embedding = tf.keras.layers.Embedding(VOCAB_SIZE,63,input_length = MAX_SEQ_LEN)
        self.lstm1 = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128,dropout=0.5, recurrent_dropout=0.0,return_sequences=True))
        self.lstm2 = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,dropout=0.5, recurrent_dropout=0.0,return_sequences=True))
        self.lstm3 = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32,dropout=0.5, recurrent_dropout=0.0,return_sequences=False))

    def call(self, inp, is_training=False):
        out = self.embedding(inp)
        out = self.lstm1(out)
        out = self.lstm2(out)
        out = self.lstm3(out)
        return out

class Negatives(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.embedding = tf.keras.layers.Embedding(VOCAB_SIZE,63,input_length = MAX_SEQ_LEN)
        self.lstm1 = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128,dropout=0.5, recurrent_dropout=0.0,return_sequences=True))
        self.lstm2 = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,dropout=0.5, recurrent_dropout=0.0,return_sequences=True))
        self.lstm3 = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32,dropout=0.5, recurrent_dropout=0.0,return_sequences=False))

    def call(self, inp, is_training=False):
        out = self.embedding(inp)
        out = self.lstm1(out)
        out = self.lstm2(out)
        out = self.lstm3(out)
        return out

class Scores(tf.keras.Model):
    def __init__(self):
        #print('Scores init')
        super().__init__()
        self.input_layer = tf.keras.layers.InputLayer(input_shape=(7,))
        self.dense1 = tf.keras.layers.Dense(256,activation='sigmoid')
        self.dense2 = tf.keras.layers.Dense(128,activation='sigmoid')
        self.dense3 = tf.keras.layers.Dense(32,activation='sigmoid')
        self.drop = tf.keras.layers.Dropout(0.3)
    def call(self, inp, is_training=False):
        #print('Scores call, input: ',self.dense1)
        out = self.input_layer(inp)
        #print('Scores call, out: ',out)
        
        out = self.dense1(out)
        out = self.drop(out)
        out = self.dense2(out)
        out = self.drop(out)
        out = self.dense3(out)
        return out

class FiveHeadNet(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.posi = Positives()
        self.nega = Negatives()
        self.scor = Scores()
        self.city = Cities()
        self.pos = Positions()
        #self.dense = tf.keras.layers.Dense(160,activation='sigmoid')
        self.out = tf.keras.layers.Dense(9,activation='softmax')

    def call(self, inp):
        #print(inp)
        out_p = self.posi(inp[0])
        out_n = self.nega(inp[1])
        out_s = self.scor(inp[2])
        out_c = self.city(inp[3])
        out_pos = self.pos(inp[4])
        out = tf.concat([out_p, out_n, out_s,out_c,out_pos], axis=1)
        #print(out)
        #out = self.dense(out)
        #out = self.dense(out)
        out = self.out(out)
        #print(out)

        return out

metric = tfa.metrics.FBetaScore(num_classes=9)

model = FiveHeadNet()


In [41]:
scores_train_data.shape

(48116, 6)

In [42]:
#model.load_weights('/content/model.weights')

In [ ]:
#!pip install tensorflow-addons
#import tensorflow_addons as tfa
#metric = tfa.metrics.FBetaScore(num_classes=9)
BATCH_SIZE = 100
NUM_EPOCHS = 120
model.compile(loss='categorical_crossentropy',optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),metrics=[metric,'accuracy'])

for i in range (NUM_EPOCHS):
  print('epoch',i)
  history = model.fit([posi_train_data,nega_train_data,scores_train_data,city_train_data,position_train_data],
                    target,
                    epochs=1,
                    batch_size=BATCH_SIZE,
                    verbose=1,
                    #steps_per_epoch=100
                    )
  pred_train = model.predict([posi_train_data[:7],nega_train_data[:7],scores_train_data[:7],city_train_data[:7],position_train_data[:7]])
  print(pred_train)
  print('pred/real')
  print(np.argmax(pred_train[0]),np.argmax(target[0]))
  print(np.argmax(pred_train[1]),np.argmax(target[1]))
  print(np.argmax(pred_train[2]),np.argmax(target[2]))
  print(np.argmax(pred_train[3]),np.argmax(target[3]))
  print(np.argmax(pred_train[4]),np.argmax(target[4]))
  print(np.argmax(pred_train[5]),np.argmax(target[5]))
  print(np.argmax(pred_train[6]),np.argmax(target[6]))


epoch 0
482/482 [==============================] - 167s 288ms/step - loss: 0.2075 - fbeta_score: 0.5118 - accuracy: 0.9294
[[1.5376635e-01 4.1183694e-03 1.1467759e-03 1.7798981e-02 9.8991878e-02
  4.9381416e-02 5.3150022e-01 1.3640444e-01 6.8916394e-03]
 [1.8149180e-03 1.9358216e-01 1.5770311e-03 1.8631181e-03 3.5675842e-02
  1.6613296e-01 4.4459540e-01 1.5345080e-01 1.3078324e-03]
 [4.6140817e-03 9.5532405e-01 2.0400525e-04 4.8573644e-04 2.6136914e-03
  6.7084990e-03 2.1455618e-02 4.0490660e-03 4.5452015e-03]
 [3.8610257e-02 5.3696519e-01 7.7630754e-04 1.1413769e-03 1.2172753e-03
  1.9747475e-02 6.1515355e-03 3.7455413e-01 2.0836519e-02]
 [5.4113251e-01 3.9578074e-01 1.2611601e-03 6.1508128e-04 2.6064308e-03
  1.0673466e-02 2.9360827e-02 8.7355468e-03 9.8343398e-03]
 [9.9224830e-01 4.4792084e-04 6.0752837e-04 3.8184502e-04 9.3658986e-05
  2.5217413e-04 2.2473857e-04 2.9358323e-04 5.4502296e-03]
 [5.6015130e-04 1.4294042e-04 2.0706591e-06 1.7055860e-04 1.1581387e-06
  1.0274510e-05 2.6

In [ ]:
#loss: 0.0930 - fbeta_score: 0.8129 - accuracy: 0.9680
num = 6
print(rewiews['positive'][num])
print('=====================================')
print("Entry example: {}".format(train_data[num]))

for i in train_data[num]:
  if i!=0:
    print(tokenizer.index_word[i],end = ' ')

В принципе норм.Работать можно
Entry example: [  2 523 463  30  34   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0 

In [ ]:
test_rewiews = pd.read_csv('/content/HeadHunter_test.csv', sep = ',',error_bad_lines = False)
test_rewiews = test_rewiews.fillna('')
test_data = test_rewiews['positive']
test_data

0         Свобода в выборе рабочего графика, условия труда
1        Компания делает ставки на молодых и перспективных
2                           Есть возможность для  развития
3        Хороший коллектив, размереная работа, нет пере...
4                   Гибкость в рабочем графике, коллектив.
                               ...                        
50646    Близость к дому  Возможность продвижения по ка...
50647                    Стабильность. Взаимовыручка. ДМС.
50648    Мне нравится как обустроен офис, есть места дл...
50649     Ничего не нравилось, поэтому и ушла, это рабство
50650                      График, развитие, сплоченность.
Name: positive, Length: 50651, dtype: object

In [ ]:
X_test = tokenizer.texts_to_sequences(test_data)

In [ ]:
test_data = sequence.pad_sequences(X_train, 
                                    padding='post',
                                    maxlen=MAX_SEQ_LEN)

In [ ]:
test_data

array([[ 179,    2,  293, ...,    0,    0,    0],
       [  33,  963, 3267, ...,    0,    0,    0],
       [  18,   31,   13, ...,    0,    0,    0],
       ...,
       [  37,   40,   14, ...,    0,    0,    0],
       [   6,    3,  328, ...,    0,    0,    0],
       [  22,  530, 5854, ...,    0,    0,    0]], dtype=int32)

In [ ]:
pred_test = model.predict(test_data)

In [ ]:
pd.DataFrame(pred_test).to_excel('/content/test.xlsx')

In [ ]:
pred_test

array([[3.1976318e-20, 0.0000000e+00, 0.0000000e+00, ..., 8.0724150e-02,
        7.0328278e-11, 4.3106970e-01],
       [3.1966182e-20, 0.0000000e+00, 0.0000000e+00, ..., 8.0721959e-02,
        7.0319923e-11, 4.3107769e-01],
       [3.1971843e-20, 0.0000000e+00, 0.0000000e+00, ..., 8.0728866e-02,
        7.0320569e-11, 4.3111455e-01],
       ...,
       [3.1966152e-20, 0.0000000e+00, 0.0000000e+00, ..., 8.0721885e-02,
        7.0323081e-11, 4.3107077e-01],
       [3.1964589e-20, 0.0000000e+00, 0.0000000e+00, ..., 8.0725327e-02,
        7.0321783e-11, 4.3108255e-01],
       [4.7704242e-03, 0.0000000e+00, 0.0000000e+00, ..., 0.0000000e+00,
        1.1226423e-02, 2.2983789e-01]], dtype=float32)

In [ ]:
def tostr(lis):
  max = lis[np.argmax(lis)]
  ret = [np.argmax(lis)]
  lis[np.argmax(lis)]=0
  while lis[np.argmax(lis)]>max*0.92:
    ret+=[np.argmax(lis)]
    lis[np.argmax(lis)]=0
  return ','.join(map(str, ret))
fsdfsd=[0.003162178	,0.328366071	,0	,0.044866722	,2.36975E-09	,0.046166088	,0.324614793	,0.008866519,	0.243957669]
#formatted_predictions = []
#tostr(fsdfsd)
formatted_predictions+=[tostr(fsdfsd)]
formatted_predictions

['1,6', '1,6', '1,6']

In [ ]:
formatted_predictions = []
for prediction in pred_test:
  #print(prediction)
  #print(tostr(prediction))
  formatted_predictions+=[tostr(prediction)]

In [ ]:
final = pd.DataFrame(formatted_predictions)

In [ ]:
final['review_id'] = test_rewiews['review_id']

In [ ]:
final.to_excel('/content/final.xlsx')

In [49]:
model.save_weights('/content/model.weights')